In [4]:
VIDEO_FRAMERATE = 30 #nr of frames per second
#it takes my pc 3 min 15 s with framerate 0.5. For 25 fps, estimated is 2 h and 45 mins
PATH_TO_VIDEO = '2022-06-14_16-38-43_S04_eye-tracking-video-world_frame.mp4'
PATH_TO_INFO_AND_CROSS = "2022-06-14_16-38-43_streamLog_actionNet-wearables_S04.hdf5"
VIDEO_ID = "S04_01"
FRAMES_SAVE_PATH = "actionNet/" + VIDEO_ID
CALIBRATION_SHIFT = 14*60 + 21 # the first part of the video is usless
PIKLE_PATH = "actionNet/S04_01"

VIDEO_SAMPLING_RATE = 1/VIDEO_FRAMERATE
import h5py, numpy as np
import cv2
import os

## Video transformation into frames

In [6]:
directory = FRAMES_SAVE_PATH

# Check if the directory exists
if not os.path.exists(directory):
    # If it doesn't exist, create it
    os.makedirs(directory)

In [ ]:
#you need to have ffmpeg exe somewhere
%%bash

FFMPEG_PATH="C:/Users/NonAv/Desktop/ffmpeg-6.0-essentials_build/bin/ffmpeg.exe"
PATH_TO_VIDEO="./2022-06-14_16-38-43_S04_eye-tracking-video-world_frame.mp4"
FRAMES_SAVE_PATH="actionNet/S04_01"

$FFMPEG_PATH -ss 00:14:21 -i $PATH_TO_VIDEO $FRAMES_SAVE_PATH/img_%010d.jpg -vf "scale=456:256" 

In [19]:
vidcap = cv2.VideoCapture(PATH_TO_VIDEO)
video_fps = vidcap.get(cv2.CAP_PROP_FPS)
framecount = vidcap.get(cv2.CAP_PROP_FRAME_COUNT)
"number of frames expected: " + str(round(framecount - video_fps* CALIBRATION_SHIFT))

'number of frames expected: 83225'

In [23]:
import os
files = os.listdir(FRAMES_SAVE_PATH)
i = 1

for file in files:
    new_file_num = int(file.split("_")[1].split(".")[0])-1
    new_file_name = "img_"+f'{new_file_num:010d}'+".jpg"
    os.rename(os.path.join(FRAMES_SAVE_PATH, file), os.path.join(FRAMES_SAVE_PATH,new_file_name))

    #i = i+1

## Labels and dataframe (records)

In [9]:



h5_file = h5py.File(PATH_TO_INFO_AND_CROSS)

device_name = 'experiment-activities'
stream_name = 'activities'

# Get the timestamped label data.
# As described in the HDF5 metadata, each row has entries for ['Activity', 'Start/Stop', 'Valid', 'Notes'].
activity_datas = h5_file[device_name][stream_name]['data']
activity_times_s = h5_file[device_name][stream_name]['time_s']
activity_times_s = np.squeeze(np.array(activity_times_s))  # squeeze (optional) converts from a list of single-element lists to a 1D list
# Convert to strings for convenience.
activity_datas = [[x.decode('utf-8') for x in datas] for datas in activity_datas]

# Combine start/stop rows to single activity entries with start/stop times.
#   Each row is either the start or stop of the label.
#   The notes and ratings fields are the same for the start/stop rows of the label, so only need to check one.
exclude_bad_labels = True # some activities may have been marked as 'Bad' or 'Maybe' by the experimenter; submitted notes with the activity typically give more information
activities_labels = []
activities_start_times_s = []
activities_end_times_s = []
activities_ratings = []
activities_notes = []
for (row_index, time_s) in enumerate(activity_times_s):
  label    = activity_datas[row_index][0]
  is_start = activity_datas[row_index][1] == 'Start'
  is_stop  = activity_datas[row_index][1] == 'Stop'
  rating   = activity_datas[row_index][2]
  notes    = activity_datas[row_index][3]
  if exclude_bad_labels and rating in ['Bad', 'Maybe']:
    continue
  # Record the start of a new activity.
  if is_start:
    activities_labels.append(label)
    activities_start_times_s.append(time_s)
    activities_ratings.append(rating)
    activities_notes.append(notes)
  # Record the end of the previous activity.
  if is_stop:
    activities_end_times_s.append(time_s)

In [10]:
import pandas as pd
action_net = pd.read_pickle("./action-net/ActionNet_train.pkl")
action_dict = {}
for i,row in action_net[["description", "labels"]].iterrows():
    desc = row[0]
    label = row[1]
    action_dict[desc] = label

In [11]:
labels_dict = {}
i=0
for label in set(action_net["labels"]):
    labels_dict[label] = i
    i+=1

{'Clean',
 'Clear',
 'Get/Put',
 'Load',
 'Open/Close',
 'Peel',
 'Pour',
 'Set',
 'Slice',
 'Spread',
 'Stack',
 'Unload'}

In [12]:
#TODO decide if we want to merge the same actions into one
from datetime import datetime
import pandas as pd

#seeding uuid 
import uuid
import random
rd = random.Random()
rd.seed(9341)

#"start_timestamp", "stop_timestamp"
records = []
shift = datetime.utcfromtimestamp(min(activities_start_times_s))
for i, label in enumerate(activities_labels) :
    #activities_end_times_s
    #start_timestamp = datetime.timestamp(datetime.utcfromtimestamp(activities_start_times_s[i]) - shift)
    #stop_timestamp = datetime.timestamp(datetime.utcfromtimestamp(activities_end_times_s[i]) - shift)
    start_frame = int((datetime.utcfromtimestamp(activities_start_times_s[i]) - shift).total_seconds() * VIDEO_FRAMERATE)
    stop_frame = int((datetime.utcfromtimestamp(activities_end_times_s[i]) - shift).total_seconds() * VIDEO_FRAMERATE)
    narration = label
    verb = action_dict[label]
    verb_class = labels_dict[verb]
    uuid_str = uuid.UUID(int=rd.getrandbits(128))
    records.append([uuid_str, VIDEO_ID, start_frame,stop_frame, narration, verb, verb_class])

records_pd = pd.DataFrame(records,columns=["uuid", "video_id","start_frame", "stop_frame", "narration", "verb", "verb_class"])

In [38]:
records_pd

,uuid,video_id,start_frame,stop_frame,narration,verb,verb_class
0,d4f63e5e-f85c-fc3f-02dc-c80aca18b35b,S04_01,0,2262,Get/replace items from refrigerator/cabinets/d...,Get/Put,3
1,c6f5c839-708e-c4cd-8b48-95cd7a4af83f,S04_01,2679,4111,Peel a cucumber,Peel,2
2,c7908761-b083-8833-13f4-16386b9d1262,S04_01,4631,5715,Peel a cucumber,Peel,2
3,829b6cb8-2528-d8bc-1f21-803cea2c3d63,S04_01,6041,7412,Peel a cucumber,Peel,2
4,e6fdddf6-dd65-2e46-cbb5-6972bfdcadf2,S04_01,7838,9082,Clear cutting board,Clear,11
5,50412c8a-9113-8423-ad35-dbd12fabac19,S04_01,9395,10835,Slice a cucumber,Slice,7
6,ac0db17b-3e92-dcbf-2ae1-800616851c99,S04_01,11362,12726,Slice a cucumber,Slice,7
7,a43e5e1c-f092-ea92-5f95-5aea0176e195,S04_01,13082,14226,Slice a cucumber,Slice,7
8,a6543ec7-188c-8f05-bfa8-51806d0ccf9b,S04_01,14779,16113,Clear cutting board,Clear,11
9,4c652a9d-9ca8-f23d-57c8-8a91ee463283,S04_01,16413,18495,Get/replace items from refrigerator/cabinets/d...,Get/Put,3


In [30]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(records_pd, test_size=0.2, random_state=9341)

In [31]:
train_final = train.reset_index(drop=True)
test_final = test.reset_index(drop=True)
train_final

,uuid,video_id,start_frame,stop_frame,narration,verb,verb_class
0,30426719-ce60-a9b4-e5f1-7695b7ada320,S04_01,52867,53092,Clean a plate with a towel,Clean,0
1,85d1a77f-bdf0-abb3-9850-056deb64012b,S04_01,27511,28235,Slice a potato,Slice,7
2,a9592c7c-c553-b5a5-5edd-37b5540ae18a,S04_01,64348,65710,"Stack on table: 3 each large/small plates, bowls",Stack,8
3,3db75ddb-4e5a-086f-fcfc-acc24eb70b85,S04_01,35114,37066,Get/replace items from refrigerator/cabinets/d...,Get/Put,3
4,2ae161ff-e4c0-0fc6-f2ef-8211b5ce1576,S04_01,33736,34551,Clear cutting board,Clear,11
5,4c652a9d-9ca8-f23d-57c8-8a91ee463283,S04_01,16413,18495,Get/replace items from refrigerator/cabinets/d...,Get/Put,3
6,2f8b4ec1-eea9-ac1d-a6fb-a71891cd9f62,S04_01,21294,22507,Peel a potato,Peel,2
7,5ad3709a-4c57-1f65-4dbe-a189bfc011a5,S04_01,53549,53732,Clean a pan with a sponge,Clean,0
8,3e93bf8f-96b5-7d34-c992-8c45d6fe4c83,S04_01,60881,63555,"Set table: 3 each large/small plates, bowls, m...",Set,1
9,a35aaa3f-5c9a-93f9-fcd4-f72fb449efc2,S04_01,54585,54763,Clean a pan with a towel,Clean,0


In [37]:
train_final.to_pickle(PIKLE_PATH + "_train.pkl")
test.to_pickle(PIKLE_PATH + "_test.pkl")